### TAREFA 12 - METÓDOS NUMÉRICOS II

Membros:
* Isaac Miller
* Milton Cassul

Tarefa descrita no final das aula [Aula#22](https://drive.google.com/file/d/1yw20ppYUfrdh0s7ws1Z24Jy-l77loZgb/view)

In [1]:
import numpy as np
np.set_printoptions(precision=5,suppress=True)

In [2]:
def MetodoQR(A,tridiag = False,epsilon = 1E-8):
    n = A.shape[0]
    val=100
    P=np.eye(n)

    if tridiag:
        decomposicao = decomposicaoQRH
    else:
        decomposicao = decomposicaoQR

    A_ant=A[:]
    #print("A=", A)

    while(val>epsilon):
        Q, R = decomposicao(A_ant, n)
        A = R.dot(Q)                    #Transf. de similaridade QtAQ 
        A_ant = A[:]
        P = P.dot(Q)
        val = Soma(A, n)
        print(f'Matriz da iteracao QR = {A} \n')

    aut = A
    return P, aut

In [3]:
def Soma(A, n):
    soma=0
    for i in range(n):
        for j in range(i):
            soma=soma+(A[i,j])**2
    return soma

In [4]:
def decomposicaoQR(A, n):
    QT = np.eye(n)
    R_ant = A
    for j in range(n-1):#coluna
        for i in range(j+1, n):#linha
            J = matriz_jacobi(R_ant, i, j, n)
            R = J@(R_ant)                       #R = QtA
            R_ant = R[:]
            QT = J@(QT)                         #Prod. matrizes Jacobi

    return QT.transpose(), R

In [5]:
def decomposicaoQRH(A, n):
    QT = np.eye(n)
    R_ant = A
    for j in range(n-1):#coluna
        J = matriz_jacobi(R_ant, j+1, j, n)
        R = J@(R_ant)
        R_ant = R[:]
        QT = J@(QT)

    return QT.transpose(), R


In [6]:
def matriz_jacobi(A,i,j,n):
    e = 10E-15
    J = np.eye(n)
    if abs(A[i,j]) <= e:
        return J
    if abs(A[j,j]) <= e: 
        if A[i,j] < 0:
            theta = np.pi/2
        else:
            theta = -np.pi/2
    else:
        theta = np.arctan(-A[i,j]/A[j,j])
         
    J[i,i] = np.cos(theta)
    J[j,j] = np.cos(theta)
    J[i,j] = np.sin(theta)
    J[j,i] = -np.sin(theta)
    
    return J

In [7]:
def householder(A):
    n = A.shape[0]
    H = np.eye(n)
    A_ant = A
    
    for i in range(n-2):
        H_i = matriz_householder(A_ant, i)
        A = H_i.T.dot(A_ant).dot(H_i)
        A_ant = A
        H = H.dot(H_i)
        
    #print('Matriz tridiagonal A_barra = \n',A,'\n')
    #print("Matriz acumulada H de householder = \n",H,'\n')
    return A, H

In [8]:
def matriz_householder(A, i):
    n = A.shape[0]
    w,z = np.zeros(n),np.zeros(n)                  #w e w'
    w[i+1:n] = A[i+1:n,i]
    z[i+1] = np.linalg.norm(w) 
    N = w - z
    N /= np.linalg.norm(N)
    
    H = np.eye(n) - (2*np.outer(N,N))
    #print("Matriz de householder = ",H,'\n')
    return H

In [9]:
if __name__ == "__main__":
    n = 5
    A = np.array([40,8,4,2,1,8,30,12,6,2,4,12,20,1,2,2,6,1,25,4,1,2,2,4,5]).reshape(n,n)
        
    print(f"Matriz a ser usada no metodo QR \n {A} \n")
    P, autv= MetodoQR(A,tridiag=False)

    print(f"Matriz diagonal A_barra = {autv} \n")
    print(f'Matriz acumulada P = {P} \n')
    auto_valores = np.diag(autv)
    print('Pares Autovalores e Autovetores \n')
    for i in range(n):
        print(f'{auto_valores[i]:.5f} - {P[:,i]} ')

    print('\nAutovetores e Autovalores de A pelo método de Jacobi(Tarefa 14): \n')
    print('Autovalor = 49.38315 --- Autovetor = [0.69806 0.57987 0.34487 0.22686 0.07784] ')
    print('Autovalor = 31.31147 --- Autovetor = [-0.70728  0.5012   0.32981  0.36252  0.0914]') 
    print('Autovalor = 11.64243 --- Autovetor = [ 0.04133 -0.59192  0.77449  0.15537  0.15475]') 
    print('Autovalor = 23.64808 --- Autovetor = [ 0.10329 -0.24728 -0.39382  0.87054  0.12351]') 
    print('Autovalor = 4.01488 --- Autovetor = [-0.0091   0.03207 -0.13179 -0.18745  0.97282] \n\n') 

    print('Metodo QR recebendo a matriz tridiagonal\n')
    A_tridiag, H = householder(A)
    P, autv = MetodoQR(A_tridiag,tridiag=True)                             #Sinalizando que está mandando uma tridiagonal
    print(f'Matriz P de depois da tridiagonalização\n {P} \n')
    HP = H.dot(P)
    print(f'Autovetores de A pelo por HP\n {HP}')


Matriz a ser usada no metodo QR 
 [[40  8  4  2  1]
 [ 8 30 12  6  2]
 [ 4 12 20  1  2]
 [ 2  6  1 25  4]
 [ 1  2  2  4  5]] 

Matriz da iteracao QR = [[44.02671  8.12341  1.17441  1.23718  0.10036]
 [ 8.12341 34.84193  4.15842  4.26685  0.22169]
 [ 1.17441  4.15842 14.14745 -3.5627   0.25292]
 [ 1.23718  4.26685 -3.5627  22.93121  0.65023]
 [ 0.10036  0.22169  0.25292  0.65023  4.0527 ]] 

Matriz da iteracao QR = [[46.78821  6.32816  0.08252  0.55678  0.00901]
 [ 6.32816 33.4969   0.39344  2.34231  0.02289]
 [ 0.08252  0.39344 16.63198 -5.93791  0.06216]
 [ 0.55678  2.34231 -5.93791 19.06495  0.15443]
 [ 0.00901  0.02289  0.06216  0.15443  4.01795]] 

Matriz da iteracao QR = [[48.24697  4.38683 -0.11042  0.18564  0.00077]
 [ 4.38683 32.29032 -0.61453  1.07538  0.00258]
 [-0.11042 -0.61453 20.66648 -5.30122  0.0141 ]
 [ 0.18564  1.07538 -5.30122 14.78101  0.04951]
 [ 0.00077  0.00258  0.0141   0.04951  4.01522]] 

Matriz da iteracao QR = [[48.9097   2.88692 -0.08213  0.04971  0.00006]
